## 1st Class: Recommendation Heuristics

> Date: June 23, 2020

- Here we are going to make a movies classifier, recommending movies to the user based on what he/she watched.
- In this context, we will use a lot the main libraries into the machine learning with Python: Pandas, Numpy, SKLearn, Matplotlib and others. 

In [1]:
import pandas as pd

movies = pd.read_csv("ml-latest-small/movies.csv")
ratings = pd.read_csv("ml-latest-small/ratings.csv")

### Translation to portuguese

If you want, run the cell below to rename the columns to be in portuguese/brazilian/pt-BR. In this case, you'll need to adapt the code to this language and uncomment the lines of this notebook.

#### Attention
Don't run the next cell if you want continue using the columns in english. In this case, let the lines commented, as they are.

In [2]:
# movies.columns  = ["filmeId", "titulo", "genero"]
# ratings.columns = ["usuarioId", "filmeId", "genero", "tempo"]

In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
movies = movies.set_index("movieId")
print(len(movies))
movies.head()

9742


,title,genres
movieId,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


In [5]:
# ratings = ratings.set_index("movieId")
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


#### First Attempt
- Considering that we don't know what kind of movie the user likes, we need to define the firsts films to suggest. 
- As we can see, we have same informations that say to us the films with more number of ratings. We can use this to make a sugestion of the most seen films into our catalog.

In [6]:
qnt_votes = ratings['movieId'].value_counts()
qnt_votes.head()

356     329
318     317
296     307
593     279
2571    278
Name: movieId, dtype: int64

In [7]:
print(movies.loc[356], "\n")
print(movies.loc[318])

title          Forrest Gump (1994)
genres    Comedy|Drama|Romance|War
Name: 356, dtype: object 

title     Shawshank Redemption, The (1994)
genres                         Crime|Drama
Name: 318, dtype: object


In [8]:
movies["qnt_votes"] = qnt_votes
movies.head()

,title,genres,qnt_votes
movieId,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0
2,Jumanji (1995),Adventure|Children|Fantasy,110.0
3,Grumpier Old Men (1995),Comedy|Romance,52.0
4,Waiting to Exhale (1995),Comedy|Drama|Romance,7.0
5,Father of the Bride Part II (1995),Comedy,49.0


In [9]:
movies.sort_values('qnt_votes', ascending = False).head(20)

,title,genres,qnt_votes
movieId,,,
356,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,279.0
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,278.0
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,251.0
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,238.0
110,Braveheart (1995),Action|Drama|War,237.0
589,Terminator 2: Judgment Day (1991),Action|Sci-Fi,224.0


### Second Attempt

#### Sort by rate

- Trying to find a better way to suggest some movies to someone, we can make our second attempt: sort by rate.
- In this case, we will need to pay attention in 2 things: the rate and the number of votes. if the rate is 5.0, but the quantity of votes is one, we have to consider that probably it isn't a movie that many people would like to see.

In [12]:
rates = ratings.groupby('movieId').mean()['rating']
rates.head()

movieId
1    3.920930
2    3.431818
3    3.259615
4    2.357143
5    3.071429
Name: rating, dtype: float64

In [13]:
movies["rating"] = rates
movies.sort_values('rating', ascending = False).head(10)

,title,genres,qnt_votes,rating
movieId,,,,
88448,Paper Birds (Pájaros de papel) (2010),Comedy|Drama,1.0,5.0
100556,"Act of Killing, The (2012)",Documentary,1.0,5.0
143031,Jump In! (2007),Comedy|Drama|Romance,1.0,5.0
143511,Human (2015),Documentary,1.0,5.0
143559,L.A. Slasher (2015),Comedy|Crime|Fantasy,1.0,5.0
6201,Lady Jane (1986),Drama|Romance,1.0,5.0
102217,Bill Hicks: Revelations (1993),Comedy,1.0,5.0
102084,Justice League: Doom (2012),Action|Animation|Fantasy,1.0,5.0
6192,Open Hearts (Elsker dig for evigt) (2002),Romance,1.0,5.0


#### Filtering the data

In [14]:
movies015p = movies.query('qnt_votes > 0.15 * qnt_votes.max()').sort_values('rating', ascending = False)
movies015p.head(10)

,title,genres,qnt_votes,rating
movieId,,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
858,"Godfather, The (1972)",Crime|Drama,192.0,4.289062
2959,Fight Club (1999),Action|Crime|Drama|Thriller,218.0,4.272936
1276,Cool Hand Luke (1967),Drama,57.0,4.271930
750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,97.0,4.268041
904,Rear Window (1954),Mystery|Thriller,84.0,4.261905
1221,"Godfather: Part II, The (1974)",Crime|Drama,129.0,4.259690
48516,"Departed, The (2006)",Crime|Drama|Thriller,107.0,4.252336
1213,Goodfellas (1990),Crime|Drama,126.0,4.250000


In [15]:
print("Minimum number of rates: ", (0.15 * movies.qnt_votes.max()))

Minimum number of rates:  49.35


## 2nd Class: Recommendation System

> Date: June 24, 2020

**Colaborative Filtering:** Filter using users ratings and comments

**Content Filtering:** Filter based on the content of the movies (genres, author, movie's casting etc.)

Now, we have users that watched some movie of the list. What can we do with this information?

In [16]:
watched_movies = [2571, 260, 480,  527, 1,  1196, 4993] # These are the movies that I watched haha ;D

movies.loc[watched_movies]

,title,genres,qnt_votes,rating
movieId,,,,
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,278.0,4.192446
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,251.0,4.231076
480,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,238.0,3.750000
527,Schindler's List (1993),Drama|War,220.0,4.225000
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0,3.920930
1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi,211.0,4.215640
4993,"Lord of the Rings: The Fellowship of the Ring,...",Adventure|Fantasy,198.0,4.106061


### 1st Attempt: Suggest movies of the same genre

If we take one of this movies and see its genres, we can use that to know similar movies that can be interesting to the people, no?

Obs.: The argument `errors = 'ignore'` is used because some movies into the list of watched movies are not into the list of `genres == "Action|Adventure|Sci-Fi"`. Therefore, 

In [17]:
act_adv_scifi_movies = movies015p.query('genres == "Action|Adventure|Sci-Fi"').drop(watched_movies, errors = 'ignore').sort_values('rating', ascending = False)

## 3rd Class: Distance between users

> Date: June 24, 2020

Using the user ratings, we can find the relationship between them and consider that if both like a specific movie, we can recommend films that the other likes.

**Example:** If I liked either X-Men and Avengers and you liked X-Men too, we can recommend Avengers to you.

-------------

**Norm:** Calculate the distance between two points 

In [18]:
# Take ratings of one user with movieId and ratings
# Define movieId as the index
# def function that do all this things
# join data of 2 users in one dataframed
# Remove lines with NaN
# Distance between 2 users

In [85]:
ratings[300:400]

,userId,movieId,rating,timestamp
300,4,21,3.0,986935199
301,4,32,2.0,945173447
302,4,45,3.0,986935047
303,4,47,2.0,945173425
304,4,52,3.0,964622786
...,...,...,...,...
395,4,1391,1.0,945173766
396,4,1449,5.0,945079767
397,4,1466,4.0,945173492
398,4,1500,4.0,964622830


In [123]:
import numpy as np
from numpy.linalg import norm

user = 'userId'
number1 = 1
number2 = 1

user1_test = ratings.query('{0} == {1}'.format(user, number1))[['rating', 'movieId']].set_index('movieId')
user2_test = ratings.query('{0} == {1}'.format(user, number2))[['rating', 'movieId']].set_index('movieId')

result_df = user1_test.join(user2_test, lsuffix = number1, rsuffix = number2).dropna()
print(result_df['rating' + '%d' % number2])
print(norm(result_df['rating' + '%d'% number1] - result_df['rating' + '%d'% number2]))

         rating1  rating1
movieId                  
1            4.0      4.0
3            4.0      4.0
6            4.0      4.0
47           5.0      5.0
50           5.0      5.0
...          ...      ...
3744         4.0      4.0
3793         5.0      5.0
3809         4.0      4.0
4006         4.0      4.0
5060         5.0      5.0

[232 rows x 2 columns]
0.0


In [202]:
from numpy.linalg import norm

def dist_between_users(data, column_users, column_ratings, column_id, user1, user2, min_data = 5, lt_min_dist = 30):
    data_user1 = data.query('{0} == {1}'.format(column_users, user1))[[column_ratings, column_id]].set_index([column_id])
    data_user2 = data.query('{0} == {1}'.format(column_users, user2))[[column_ratings, column_id]].set_index([column_id])
    data_users = data_user1.join(data_user2, lsuffix = user1, rsuffix = user2).dropna()
#     print(data_users)
    if(len(data_users) < min_data):
        distance = lt_min_dist
    else:
        distance = norm(data_users[column_ratings + '%d' % user1] - data_users[column_ratings + '%d' % user2])
    return distance

In [203]:
dist_between_users(ratings, 'userId', 'rating', 'movieId', 1, 77)

0.0

In [204]:
norm_1_4 = dist_between_users(ratings, 'userId', 'rating', 'movieId', 1, 4)
norm_1_3 = dist_between_users(ratings, 'userId', 'rating', 'movieId', 1, 3)
print(norm_1_4)
print(norm_1_3)
norm_1_1 = dist_between_users(ratings, 'userId', 'rating','movieId', 1, 1)
print(norm_1_1)

11.135528725660043
8.200609733428363
0.0


In [205]:
from math import sqrt

def pitagoras(num1, num2):
    return (sqrt((pow(num1, 2) + pow(num2, 2))))


print(pitagoras(4, 5))
print(norm([4, 5]))

6.4031242374328485
6.4031242374328485


In [206]:
# Show number of user
# Show array of user (show user only one time)
# For loop to pass through each user
# Execute function to calculate distance between one user and all the others
# Make a dataFrame with 3 columns: Users, other_users, distance

In [207]:
ratings.userId.unique()[1:20]

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20])

In [208]:
qnt_users = len(ratings.userId.unique())
print(qnt_users)

610


In [209]:
def distance_of_others(data, column_users, column_ratings, columnId, main_user):
    qnt_users = len(data[column_users].unique())
    dist_user = [[main_user, (user + 1), dist_between_users(ratings, column_users, column_ratings, columnId, main_user, (user + 1))] for user in range(qnt_users)]
    dist_user_df = pd.DataFrame(data = dist_user, columns = ['main_user', 'other_user', 'distance'])
    dist_user_df = dist_user_df.sort_values('distance').set_index('other_user').drop(main_user)
    return dist_user_df

dist_user_1 = distance_of_others(ratings, "userId", "rating", 'movieId', main_user)

In [210]:
dist_user_1.head(10)

,main_user,distance
other_user,,
77,1,0.000000
511,1,0.500000
366,1,0.707107
523,1,1.000000
49,1,1.000000
9,1,1.000000
258,1,1.000000
319,1,1.118034
398,1,1.224745


In [211]:
dist_user_1[500:550]

,main_user,distance
other_user,,
292,1,11.629703
287,1,11.683321
219,1,11.926860
42,1,12.000000
132,1,12.247449
89,1,12.389512
330,1,12.389512
469,1,12.409674
156,1,12.409674
